In [1]:
#Import modules

import pandas as pd
# from IPython.display import display, HTML
# from numpy import log10, nan, inf, random, clip, arange

# path = './../export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

# display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib inline

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

from numpy import arange, log10

from functions import chunk, agg, finalize
tunique = dd.Aggregation('tunique', chunk, agg,finalize)
first = dd.Aggregation('first', chunk, agg,finalize)

In [3]:
# dd.read_csv('2014-*.csv')

# drive_path = './../export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'
drive_path = './../../../../media/miglesia/Elements/export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

n_bins = 20
df_list = n_bins * [ 0 ]

for dataset_i in range(n_bins):
    df_list[dataset_i] = dd.read_csv(drive_path+'/samplings/YMxpb_size20'+str(dataset_i).zfill(2)+'.csv',
                                    usecols = ['YEAR','ID','VAT','VART_sum'])
    
    

In [4]:
data = dd.concat(df_list)
data = data.groupby(['YEAR','ID','VAT'])['VART_sum'].sum().reset_index()

In [5]:
with ProgressBar():
    out = data.compute()

data = dd.from_pandas(out, npartitions = 100)

[########################################] | 100% Completed |  4min 57.9s


In [ ]:
# firm_sizes = log10(data.groupby(['ID', 'YEAR'])['VART_sum'].sum().reset_index().groupby('ID')[['VART_sum']].mean().compute())

In [11]:
data.compute().to_csv('buyer_seller_links.csv', index = False)

In [ ]:
# window = 3
# assortativity_res = []
ID_degree_res = []
VAT_degree_res = []

for window in [5, 1, 3]:
    gap = (window-1)/2
    center_years = arange(1997, 2014, 2)
    print window

    for Yc in center_years:
        print Yc
        data_sec = data.loc[data.YEAR - Yc <= gap]
#         data_sec.groupby(['ID', 'VAT']).agg({'CN ID 8': tunique, 'VART_sum': sum })

        data_sec_by_ID = data_sec.groupby(['ID']).agg({'VAT': tunique, 'VART_sum': sum})

        ID_degree = data_sec_by_ID[['VAT']].reset_index()
        ID_degree.columns = [u'ID', u'ID_degree']
        ID_degree['center_year'] = Yc
        ID_degree['window'] = window
        
        with ProgressBar():
            ID_deg = ID_degree.compute()
            ID_deg['bin'] = pd.cut(log10(ID_deg['ID_degree']), bins = arange(-.49, 5.99, .25))
            ID_deg.to_csv('ID_deg_'+str(Yc)+'_'+str(window))
#         ID_degree_res += [ID_degree]     

#         ID_deg = pd.read_csv()
        sampling = ID_deg.groupby(['bin'], observed = True).apply(lambda x: x.sample(200, replace = True))

        data_sec_sample = data_sec.loc[data_sec.ID.isin(sampling['ID'].values)]
        data_sec_by_VAT = data_sec_sample.groupby(['VAT']).agg({'ID': tunique, 'VART_sum': sum})

        VAT_degree = data_sec_by_VAT[['ID']].reset_index()
        VAT_degree.columns = [u'VAT', u'VAT_degree']
        VAT_degree['center_year'] = Yc
        VAT_degree['window'] = window
        VAT_degree_res += [VAT_degree]
        with ProgressBar():
            VAT_deg = VAT_degree.compute()
            VAT_deg.to_csv('VAT_deg_save_'+str(Yc)+'_'+str(window))

5
1997
[###############                         ] | 37% Completed | 22.9s

In [ ]:
VAT_degree_res = []
for window in [1, 3, 5]:
    gap = (window-1)/2
    center_years = arange(1997, 2014, 2)
    print window

    for Yc in center_years:
    
        ID_deg = pd.read_csv('./ID_deg_save.csv')
        ID_deg = ID_deg.loc[(ID_deg.center_year == Yc) & (ID_deg.window == window)]
        ID_deg['bin'] = pd.cut(log10(ID_deg['ID_degree']), bins = arange(-.49, 5.99, .25))
        sampling = ID_deg.groupby(['bin'], observed = True).apply(lambda x: x.sample(50, replace = True))

        data_sec_sample = data_sec.loc[data_sec.ID.isin(sampling['ID'].values)]
        data_sec_by_VAT = data_sec_sample.groupby(['VAT']).agg({'ID': tunique, 'VART_sum': sum})

        VAT_degree = data_sec_by_VAT[['ID']].reset_index()
        VAT_degree.columns = [u'VAT', u'VAT_degree']
        VAT_degree['center_year'] = Yc
        VAT_degree['window'] = window
        VAT_degree_res += [VAT_degree]
        with ProgressBar():
            VAT_deg = VAT_degree.compute()
            VAT_deg.to_csv('VAT_deg_save_'+str(Yc)+'_'+str(window))

#         assortativity_info = data_sec_sample.groupby(['ID', 'VAT'])[['VART_sum']].mean().reset_index().merge(
#             ID_degree).merge(VAT_degree)
#         assortativity_res += [assortativity_info]


1
1997
[########################################] | 100% Completed |  2min  2.6s
1999
[########################################] | 100% Completed |  3min 33.6s
2001
[########################################] | 100% Completed |  5min 35.0s
2003
[########################################] | 100% Completed |  6min 59.7s
2005
[########################################] | 100% Completed |  9min 48.0s
2007
[########################################] | 100% Completed | 12min 26.0s
2009
[########################################] | 100% Completed | 15min  1.5s
2011
[########################################] | 100% Completed | 18min 18.1s
2013
[########################################] | 100% Completed | 18min  7.1s
3
1997
[########################################] | 100% Completed |  2min 16.9s
1999
[########################################] | 100% Completed |  3min 23.8s
2001
[########################################] | 100% Completed |  5min 24.9s
2003
[########################################] 

In [ ]:
# ID_degree['bin'] = 
# pd.cut(log10(ID_degree['ID_degree']), bins = arange(-.49, 5.99, .5))

In [ ]:
# ID_deg_res = dd.concat(ID_degree_res)
# with ProgressBar():
#     ID_deg = ID_deg_res.compute()

VAT_deg_res = dd.concat(VAT_degree_res)
with ProgressBar():
    VAT_deg = VAT_deg_res.compute()

In [ ]:
# ID_deg.to_csv('./ID_deg_save.csv', index = False)
VAT_deg.to_csv('./VAT_deg_save.csv', index = False)

In [ ]:
plt.hist(log10(ID_deg['ID_degree']), bins = arange(-.49, 5.99, .25))
plt.show()

In [ ]:
# 
# data_sec_sample.merge(ID_deg).merge(VAT_deg).compute()
assortativity_info = data_sec_sample.groupby(['ID', 'VAT'])[['VART_sum']].mean().reset_index().merge(
        ID_degree).merge(VAT_degree)

In [ ]:

with ProgressBar():
    assortativity_df = assortativity_info.compute()

In [ ]:
assortativity_info.groupby(pd.cut(log10(assortativity_info['ID_degree']), bins = arange(-.49, 5.99, .5)))#['VAT_degree'].describe()

In [ ]:
ID_deg['bin'] = pd.cut(log10(ID_deg['ID_degree']), bins = arange(-.49, 5.99, .5))
ID_deg.groupby(['center_year','bin'])

In [ ]:
data_sec_sample

In [ ]:
# ID_deg.to_csv('degree_values.csv', index = False)

from numpy import power
left = power(10, arange(-.1, 5, .2)).round(1)
right = power(10, arange(-.1, 5, .2) + .2).round(1)
bins = pd.IntervalIndex.from_arrays(left, right)

ID_deg_part = ID_deg.loc[(ID_deg.center_year == Yc) & (ID_deg.window == window)]
ID_deg_part.groupby(pd.cut(ID_deg_part['ID_degree'], bins), observed = True).count().sort_index()

sample_IDs = ID_deg_part.groupby(pd.cut(ID_deg_part['ID_degree'], bins), observed = True).apply(lambda x: x.sample(1000, replace = True))['ID'].values

data_sec = data.loc[(data.YEAR - Yc <= gap) & (data.ID.isin(sample_IDs))]

    
#.apply(lambda x: x.sample(5000, replace = True))['ID'].values

In [ ]:
summary_results = []

for df_degrees in results:

    with ProgressBar():
        x = df_degrees.compute()

    summary_result = x.groupby(pd.cut(x['ID_degree'], bins)).agg({'VART_sum': 'sum', 'ID_degree': 'mean',
                                            'VAT_degree': 'describe', 'center_year' : 'median', 'window': 'median'})
    summary_results += [summary_result]

In [ ]:
pd.concat(summary_results).dropna().to_csv('./assortativity_2000_full.csv')

In [ ]:
pd.concat(summary_results)

In [ ]:
pd.concat(summary_results).dropna().to_csv('./assortativity_2000_8plus.csv')

In [ ]:
res = dd.concat(results, axis = 1)


with ProgressBar():
    out_2 = res.compute()

In [ ]:
fig, ax = plt.subplots(1)
df_degrees.groupby('VAT_degree_bin')['ID_degree','VAT_degree'].quantile(.25).plot(x = 'VAT_degree', y = 'ID_degree', marker = '', ax = ax)
df_degrees.groupby('VAT_degree_bin')['ID_degree','VAT_degree'].quantile(.5).plot(x = 'VAT_degree', y = 'ID_degree', marker = '', ax = ax)
df_degrees.groupby('VAT_degree_bin')['ID_degree','VAT_degree'].quantile(.75).plot(x = 'VAT_degree', y = 'ID_degree', marker = '', ax = ax)

# df_degrees.groupby('ID_nunique_bin')['VAT_nunique','ID_nunique'].mean().plot(x = 'ID_nunique', y = 'VAT_nunique', marker = 'o', ax = ax)
df_degrees.groupby('ID_nunique')['VAT_nunique'].median().plot(x = 'index', y = 'VAT_nunique', marker = '.', linewidth = 0, ax = ax)
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
# ### Choose a bin_size, 
# size_df_list = n_size_bins * [ None ]

# for s in range(14, n_size_bins):

#     bs_ix_df = exp_index.loc[exp_index.size_bins == s]

#     size_i_df_list = []
#     for dataset_i in bs_ix_df.exp_mma_cat.unique():
#         df = df_list[dataset_i]
#         size_i_df_list += [df.loc[df.ID.isin(bs_ix_df.ID)]]

#         size_df_list[s] = pd.concat(size_i_df_list)